# Machine learning tutorial at CMSDAS at CERN, June 2024
## Exercise 5: regression using a convolutional neural network

Evan Armstrong Koenig, Matthias Komm, Pietro Vischia

The core of this tutorial comes from https://github.com/vischia/data_science_school_igfae2024 (Pietro Vischia (pietro.vischia@cern.ch)).

The CMSDAS version extends it to consider a convolutional network to regress Higgs quantities, plus some fixes.


## Convolutional neural networks

When processing images, it may be convenient to take into account the fact that neighbouring pixels are usually very correlated (e.g. if one pixel is in the center of an azure eye, there is a high chance that neighbouring bins will be also azure).

The intuition by Yann LeCun (LeNet) was to use the operation of convolution to summarize the information of neighbouring bins.


$$s(t) = \int x(a) w(t-a)da$$

- When discretized, integral becomes a sum
  - $x$ input
  - $w$ kernel: specifies how far does the averaging goes
  - $s$ feature map

An illustration:

<img src="figs/conv.png" width="80%"/>


Furthermore, the topology of the image is taken into account by *parameter sharing*: the idea is that learning the same structure in different places of the image can be done efficiently if the parameters that learns that structure are the same in different portions of the network. This also introduces a further element of locality similar to the concept of *field of vision*
An illustration:

<img src="figs/conv_field.png" width="80%"/>


Further summarization (e.g. to learn the same concept under different transformations, like rotations) can be achieved by pooling:

<img src="figs/conv_pooling.png" width="80%"/>

All of this results in an architecture that can learn progressively the structural elements of an array with an image-like structure. The animation is from the original convolutional neural network from Yann LeCun (now at META), that recognized handwritten digits with unprecedented quality.

<img src="figs/asamples.gif" width="80%"/>
<img src="figs/conv_pooling.png" width="80%"/>

Acknowledging that a dataset has an image-like structure consists in deciding that the best *mathematical representation* of this object is that of an image, including e.g. concepts like *closeness*.

We will use a CNN for regression, that is we will solve the problem of regression by imposing that the data are populating an "image" in the phase space of the detector.

In [ ]:
# Uncomment and run this if you are running on Colab (remove only the "#", keep the "!").
# You can run it anyway, but it will do nothing if you have already installed all dependencies
# (and it will take some time to tell you it is not gonna do anything)


#from google.colab import drive
#drive.mount('/content/drive')
#%cd "/content/drive/MyDrive/"
#! git clone https://github.com/vischia/data_science_school_igfae2024.git
#%cd machine_learning_tutorial
#!pwd
#!ls
#!pip install livelossplot shap

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (8, 6)
matplotlib.rcParams['axes.labelsize'] = 14

import glob
import os
import re
import math
import socket
import json
import pickle
import gzip
import copy
import array
import numpy as np
import numpy.lib.recfunctions as recfunc
from tqdm import tqdm

#from scipy.optimize import newton
#from scipy.stats import norm

import uproot

import datetime
from timeit import default_timer as timer

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.tree import export_graphviz
from sklearn.inspection import permutation_importance
try:
    # See #1137: this allows compatibility for scikit-learn >= 0.24
    from sklearn.utils import safe_indexing
except ImportError:
    from sklearn.utils import _safe_indexing

import pandas as pd
import torchinfo
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.nn.functional as F

# Import data

We will use simulated events corresponding to three physics processes.

- ttH production
- ttW production
- Drell-Yan production

We will select the multilepton final state, which is a challenging final state with a rich structure and nontrivial background separation.

<img src="figs/2lss.png" alt="ttH multilepton 2lss" style="width:40%;"/>


In [ ]:
INPUT_FOLDER = '/cmsuf/data/store/user/ekoenig/cmsdas/2024/short-ex-mlg'
sig = uproot.open(os.path.join(INPUT_FOLDER,'signal.root'))['Friends'].arrays(library="pd")
bk1 = uproot.open(os.path.join(INPUT_FOLDER,'background_1.root'))['Friends'].arrays(library="pd")
bk2 = uproot.open(os.path.join(INPUT_FOLDER,'background_2.root'))['Friends'].arrays(library="pd")

In [ ]:
device = torch.device("cpu")

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
if torch.cuda.is_available() and torch.cuda.device_count()>0:
    device = torch.device("cuda")
    
print ("Available device: ",device)

# Convolutional Networks

A convolutional network specializes in learning from spatially structured data, such as images. The key idea is to use convolutional layers to learn local patterns in the data, which are then combined in deeper layers to learn more complex patterns. To be able to uses a convolutional network, we need to represent our data as 2D images.

Lets try to accomplish the regression task using a convolutional network.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

signal = sig.drop(["Hreco_Lep2_pt", "Hreco_Lep2_eta", "Hreco_Lep2_phi", "Hreco_Lep2_mass", "Hreco_evt_tag", "Hreco_HTXS_Higgs_y"], axis=1 )

X = signal.drop(["Hreco_HTXS_Higgs_pt"], axis=1)

# learn to regress the log of the pt
y = signal[["Hreco_HTXS_Higgs_pt"]].apply(np.log) 

Nevents = 20_000
X = X[:Nevents]
y = y[:Nevents]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)
print("We have", len(X_train), "training samples and ", len(X_test), "testing samples")

# scale the target variable
y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)
y_test = y_scaler.transform(y_test)

In [ ]:
# list all objects that have eta/phi coordinates
objects = [
    "Hreco_Lep0",
    "Hreco_Lep1",
    # "Hreco_HadTop",
    "Hreco_All5_Jets",
    # "Hreco_More5_Jets",
    "Hreco_Jets_plus_Lep",
]


We can build a 2D image from the eta/phi coordinates of the leptons.

In [ ]:
def digitize(x, bins):
    # digitize x into bins
    idx = np.digitize(x, bins) - 1

    # clip values outside the bins
    idx = np.clip(idx, 0, len(bins)-2)
    return idx

def build_image(X, Nbins=5, transforms=[]):
    """
    Build a 2d image of energy deposits
    """

    # using the eta and phi coordinates as x and y
    for obj in objects:
        X[f'{obj}_x'] = X[f'{obj}_eta']
        X[f'{obj}_y'] = X[f'{obj}_phi']

    # apply any transformations to the coordinates
    for transform in transforms:
        X = transform(X)

    index = np.arange(len(X))
    eta_edges = np.linspace(-5, 5, Nbins+1)
    phi_edges = np.linspace(-3.2, 3.2, Nbins+1)

    # create the image with the energy deposits
    img = np.zeros((len(X), 1, Nbins, Nbins))
    for obj in objects:
        et = np.sqrt(X[f"{obj}_pt"]**2 + X[f"{obj}_mass"]**2)
        eta_bin = digitize(X[f"{obj}_x"], eta_edges)
        phi_bin = digitize(X[f"{obj}_y"], phi_edges)
        img[index, 0, eta_bin, phi_bin] += et

    # use the log of the energy instead
    img[img > 0] = np.log(img[img > 0])
    return img

In [ ]:
# special image feature scaler that ignores the empty pixels in our images
class ImageMinMaxScaler:
    def __init__(self):
        self.max = None
        self.min = None

    def fit(self, img):
        self.max = np.array([ F[F != 0].max() for F in img.transpose(1, 0, 2, 3) ])[None, :, None, None]
        self.min = np.array([ F[F != 0].min() for F in img.transpose(1, 0, 2, 3) ])[None, :, None, None]

    def transform(self, img):
        return np.where(img != 0, (img - self.min) / (self.max - self.min), 0)

    def fit_transform(self, img):
        self.fit(img)
        return self.transform(img)


In [ ]:
scaler = ImageMinMaxScaler()

img_train = build_image(X_train, Nbins=5)
img_test = build_image(X_test, Nbins=5)


img_train = scaler.fit_transform(img_train)
img_test = scaler.transform(img_test)

print (f"Image format: {img_train.shape}")

In [ ]:
import matplotlib.colors as mcolor

norm = mcolor.Normalize(vmin=0, vmax=1)

def plot_event_image(ax, img, title):
    img = np.where(img != 0, img, np.nan)
    c = ax.imshow(img[0], norm=norm)
    ax.set_title(title)
    return c

fig, axs = plt.subplots(2, 2, figsize=[12,12])
c0 = plot_event_image(axs[0, 0], img_train[0], 'Training Event 0')
c1 = plot_event_image(axs[0, 1], img_train[1], 'Training Event 1')
c2 = plot_event_image(axs[1, 0], img_train[2], 'Training Event 2')
c3 = plot_event_image(axs[1, 1], img_train[3], 'Training Event 3')

fig.colorbar(c0, ax=axs)
plt.show()
plt.close()

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, y, device=torch.device("cpu")):
        self.X = torch.Tensor(X.values if isinstance(X, pd.core.frame.DataFrame) else X).to(device)
        self.y = torch.Tensor(y.values if isinstance(X, pd.core.frame.DataFrame) else y).to(device)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        label = self.y[idx]
        datum = self.X[idx]
        
        return datum, label
    
train_dataset = MyDataset(img_train, y_train, device=device)
test_dataset = MyDataset(img_test, y_test, device=device)

batch_size=2048
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

batch_x, batch_y = next(iter(train_dataloader))
B, C, H, W = batch_x.shape

We can use the `torch.nn.Conv2d` module to perform the convolutional operation. Unlike a dense network, a convolutional network learns a set of filters that are applied to the input images. The filters are learned during training and are used to extract local patterns from the input images.

In [ ]:
conv = nn.Conv2d(in_channels=C, out_channels=32, kernel_size=3, stride=1, device=device) # kernel_size=3 means 3x3 kernel
batch_x.shape, conv(batch_x).shape

Notice that the size of the output image is smaller than the input image. This is because the convolution operation is performed by sliding a filter over the input image. The output size is determined by the size of the input image, the size of the filter, and the stride of the convolution operation. 

Here is an example of a simple convolutional network with two convolutional layers.

In [ ]:
conv = nn.Sequential(
    nn.Conv2d(in_channels=C, out_channels=32, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.Flatten(), # flatten the 2D image into a 1D vector
).to(device=device)

B, F = conv(batch_x).shape
print(B, F)

We can then use dense neural network layers to perform the regression task.

In [ ]:
linear = nn.Sequential(
    nn.Linear(F, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.ReLU(),
).to(device=device)

linear(conv(batch_x)).shape

Putting this together into a single model, we have the following architecture:

In [ ]:
class ConvolutionNet(nn.Module):
    def __init__(self, C, H, W):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=C, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        B, F = self.conv(torch.zeros(1, C, H, W)).shape
        self.linear = nn.Sequential(
            nn.Linear(F, 1),
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.linear(x)
        return x

In [ ]:
# reinitialize our input data for training

Nbins = 5

transforms = [
    # center_objects, 
    # zoom_objects,
]

img_train = build_image(X_train, Nbins=Nbins, transforms=transforms)
img_test = build_image(X_test, Nbins=Nbins, transforms=transforms)


scaler = ImageMinMaxScaler()
img_train = scaler.fit_transform(img_train)
img_test = scaler.transform(img_test)


print (f"Image format: {img_train.shape}")
fig, axs = plt.subplots(2, 2, figsize=[12,12])
c0 = plot_event_image(axs[0, 0], img_train[0], 'Training Event 0')
c1 = plot_event_image(axs[0, 1], img_train[1], 'Training Event 1')
c2 = plot_event_image(axs[1, 0], img_train[2], 'Training Event 2')
c3 = plot_event_image(axs[1, 1], img_train[3], 'Training Event 3')

fig.colorbar(c0, ax=axs)
plt.show()
plt.close()


train_dataset = MyDataset(img_train, y_train, device=device)
test_dataset = MyDataset(img_test, y_test, device=device)

batch_size=2048
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

batch_x, batch_y = next(iter(train_dataloader))
B, C, H, W = batch_x.shape

In [ ]:
model = ConvolutionNet(C, H, W).to(device=device)

epochs=50
learningRate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
loss_fn = torch.nn.MSELoss()

def train_loop(dataloader, model, loss_fn, optimizer, scheduler, device):
    size = len(dataloader.dataset)
    losses=[] # Track the loss function
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    #for batch, (X, y) in enumerate(dataloader):
    for (X,y) in tqdm(dataloader):
        # Reset gradients (to avoid their accumulation)
        optimizer.zero_grad()
        # Compute prediction and loss
        pred = model(X)
        #if (all_equal3(pred.detach().numpy())):
        #    print("All equal!")
        loss = loss_fn(pred, y)
        losses.append(loss.detach().cpu())
        # Backpropagation
        loss.backward()
        optimizer.step()

    scheduler.step()
    return np.mean(losses)

def test_loop(dataloader, model, loss_fn, device):
    losses=[] # Track the loss function
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
        #for (X,y) in tqdm(dataloader):
            pred = model(X)
            loss = loss_fn(pred, y).item()
            losses.append(loss)
            test_loss += loss
            #correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    return np.mean(losses)

def predict_loop(dataloader, model, device):
    predictions=[]
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            predictions.append(pred)
    return torch.cat(predictions).cpu().numpy()


train_losses=[]
test_losses=[]
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss=train_loop(train_dataloader, model, loss_fn, optimizer, scheduler, device)
    test_loss=test_loop(test_dataloader, model, loss_fn, device)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print("Avg train loss", train_loss, ", Avg test loss", test_loss, "Current learning rate", scheduler.get_last_lr())
print("Done!")

In [ ]:
plt.figure()
plt.plot(train_losses[1:], label="Average training loss")
plt.plot(test_losses[1:], label="Average test loss")
plt.legend(loc="best")
plt.show()
plt.close()

In [ ]:
plt.figure()
y_pred = predict_loop(test_dataloader, model, device)
plt.scatter(y_pred, y_test, marker='o', s=1.)
plt.xlabel("Predicted value")
plt.ylabel("True value")

minim, maxim = np.min(y_test), np.max(y_test)

plt.ylim(minim, maxim)
plt.xlim(minim, maxim)
plt.plot([minim, maxim],[minim, maxim],linestyle='--',c='black')
plt.show()
plt.close()

In [ ]:
plt.figure()
diff = y_pred-y_test
hist,_,_ = plt.hist(diff, bins=100)
fractions = [2.3,15.85,50,84.15,97.7]
percentiles = np.percentile(diff,fractions)
for i in range(len(percentiles)):
    plt.plot([percentiles[i],percentiles[i]],[0,max(hist)*1.1],c='black',linestyle='--')
    plt.text(percentiles[i],max(hist)*1.11,f"{fractions[i]:.0f}%")
plt.xlabel("Predicted value - True value")
plt.show()
plt.close()

## Ways to improve

The model is not very good. Try to improve it by changing the architecture or input data

Just like scaling our input features, it is always a good idea to standardize the input images. 
* Centering the image
* Zooming in on the important parts
* Rotating the image 
* Changing the size of our image

Here are some examples on how you can do the centering and zooming 
```
def center_objects(X):
    mean_x = np.mean([X[f"{obj}_x"] for obj in objects])
    mean_y = np.mean([X[f"{obj}_y"] for obj in objects])

    for obj in objects:
        X[f"{obj}_x"] -= mean_x
        X[f"{obj}_y"] -= mean_y

    return X

def zoom_objects(X):
    std_x = np.std([X[f"{obj}_x"] for obj in objects])
    std_y = np.std([X[f"{obj}_y"] for obj in objects])

    for obj in objects:
        X[f"{obj}_x"] /= std_x
        X[f"{obj}_y"] /= std_y

    return X
```
You can add these methods to the code above and try transforming our data

These techniques can help the network learn better. We also can always modify the architecture as we did in the DNN
* add dropout layers
* use a different activation function
* add batch normalization layers
* reduce the number of layer
* reduce the number of nodes

The End